In [35]:
import json
import re
from functions import *

# Disable verification warning when accessing GSO site
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [ ]:
# # Set to True if new reports are updated
# get_report_url = False

# # Set to True to check if the values are appropriate, False to export them as csv, and combine them into one Excel file
# check_values = False

# download_path = os.path.join(os.getcwd(), "raw_xlsx")

# if get_report_url == True:
#     all_reports_url = crawl_url()
# else:
#     with open("all_reports_url.pkl", "rb") as f:
#         all_reports_url = pickle.load(f)

# test = all_reports_url[0:24]

In [ ]:
# # Set Chrome Driver Options
# options = webdriver.ChromeOptions()
# prefs = {
#     "download.default_directory" : download_path,
#     "download.prompt_for_download": False
# }
# options.add_experimental_option("prefs",prefs)
# options.binary_location = "./chrome-win64/chrome.exe"
# options.add_argument("--headless")


In [3]:
'''
There are 3 types of dataformats:
- Quarterly reports: include data of Mar, Jun, Sep, Dec
- January reports
- Other months
'''
quarterly_files = []
monthly_files = []
january_files = []

for excel_path in os.listdir(download_path):
    if any(q in excel_path for q in ["Q", "03", "06", "09", "12", "T3", "T6", "T9", "T12"]):
        quarterly_files.append(excel_path)
    elif any(m in excel_path for m in ["01", "T01"]):
        january_files.append(excel_path)
    else:
        monthly_files.append(excel_path)   

print(quarterly_files)
print(january_files)

['02-TABLE-06-2023.xlsx', '02-TABLE-06-2024.xlsx', '02-TABLE-09-2023.xlsx', '02-TABLE-12-2023.xlsx', '02.TABLE-03.2024-change.xlsx', '2.TABLE-Q1-2025.xlsx', 'TABLE-09-2024.xlsx', 'TABLE-T12-2024.xlsx']
['02-TABLE-01-2024.xlsx', '02.TABLE-01-2025.xlsx']


In [71]:
# Check if data have the same sheet names
# Quarterly data and January data files are formatted differently, so they will be separated into a different dataset
# def check_columns(name_list= [quarterly_files, monthly_files, january_files]):
#     sheet_names = []
#     for file in name_list:
#         excel = os.path.join("raw_xlsx", file)
#         sheet_names.append(pd.ExcelFile(excel).sheet_names)
#     data = pd.DataFrame(sheet_names).transpose()
#     data.columns = name_list
#     return data

# def combine_columns(df: pd.DataFrame, n_columns: int):
#     df = df.rename(columns={
#         0:"name"
#     })
#     for i in range(0, n_columns - 1):
#         df = df.assign(
#             name = lambda df : df.loc[:, "name"].combine_first(df[i+1])
#         )
#     return df

check_columns(quarterly_files)

,02-TABLE-06-2023.xlsx,02-TABLE-06-2024.xlsx,02-TABLE-09-2023.xlsx,02-TABLE-12-2023.xlsx,02.TABLE-03.2024-change.xlsx,2.TABLE-Q1-2025.xlsx,TABLE-09-2024.xlsx,TABLE-T12-2024.xlsx
0,1.GDP current,1.GDP current,1.GDP current,6NN,1.GDP,1.GDP,1.GDP current,1 GDP current
1,2.GDP compare,2.GDP compare,2.GDP compare,1 GDP current,2. Agriculture,2. Agriculture,2.GDP compare,2 GDP price 2010
2,3. Agriculture,3. Agriculture,3. Agriculture,2 GDP price 2010,3-4. Livestock forestry,3-4. Livestock forestry,3. Agriculture,3 Agriculture
3,4.winter-spring crop,4.winter-spring crop,"4,5. Livestock forestry",3 Agriculture,5. Fishery,5. Forestry,"4,5. Livestock forestry",4 Main crops
4,5.winter-spring crop rice,5. Spring crop rice,6. Fishery,4 Main crops,6.IIP,6. Fishery,6. Fishery,5 Annual industrial plants
5,"6,7 Livestock forestry","6,7 Livestock forestry",7. IIP,5 Annual industrial plants,7.Key indus products,6.IIP,7. IIP,6 Perennial industrial plants
6,8. Fishery,8. Fishery,8.IIP Q,6 Perennial industrial plants,8.Consump and invent index,7.Key indus products,8.IIP Q,7 Livestock
7,9. IIP,9. IIP,9. Industrial product,7 Livestock,9.LEI,8.Consump and invent index,9. Industrial product,8. Forestry
8,10.IIP Q,10.IIP Q,10. Industrial product Q,8-9 Forest Fishery,10.LEI province,9.LEI,10. Industrial product Q,9. Fishery
9,11. Industrial product,11. Industrial product,11. Consump and invent index,10 IIP month,11. Enterprise Indicators,10.LEI province,11. Consump and invent index,10 IIP month


In [56]:

with open("sheetnames.json") as js:
    sheet_cols = json.load(js)
sheet_cols["january"][f"{1}"]["columns"]

{'1': 'pct_mom', '2': 'pct_yoy'}

In [70]:
# Process January data
number_of_sheets = 19

with open("sheetnames.json") as js:
    sheet_cols = json.load(js)

for sheet_index in tqdm(range(0, number_of_sheets)):
    try:
        combine_df = pd.DataFrame()

        if sheet_index == 15:
            row_adj = 1
        else:
            row_adj = 0
        for excel_path in monthly_files:
            sheet = use_columns(excel_path, sheet_index).iloc[1 + row_adj:,:]
            combine_df = pd.concat([combine_df, sheet], axis = 0)

        # Custom conditions
        if sheet_index in [0, 10, 11, 13, 14, 15, 16, 17, 18]: # Sheets that need to merge multiple name columns to make sense
            if sheet_index == 15:
                ncol = 3
                bad_label = combine_df.iloc[:,1].astype(str).str.contains(r"Of which:", regex = True, flags = re.IGNORECASE)
                combine_df.loc[bad_label, 1] = None 
            else:
                ncol = 2
            combine_df = combine_columns(combine_df, ncol)
            var_list = ["month", "name"]
        else:
            var_list = ["month"]
        if sheet_index == 2:
            adjust = 1
        else:
            adjust = 0

        var_list.extend(list(sheet_cols["january"][f"{sheet_index}"]["columns"].values()))

        combine_df = combine_df.rename(columns=dict(
            list(zip(
                list(map(int, sheet_cols["january"][f"{sheet_index}"]["columns"].keys())),
                list(sheet_cols["january"][f"{sheet_index}"]["columns"].values())
            ))
        ))[var_list] # Rearranging the columns
        
    except Exception as error:
        print(error)

 68%|██████▊   | 13/19 [00:31<00:14,  2.48s/it]

'12'


 74%|███████▎  | 14/19 [00:34<00:12,  2.57s/it]

"['goods_service'] not in index"


 84%|████████▍ | 16/19 [00:38<00:07,  2.35s/it]

"['yoy_vol_1000_ton', 'yoy_val_mil_usd'] not in index"


100%|██████████| 19/19 [00:46<00:00,  2.43s/it]


In [65]:
clean_data(combine_df, var_list[1], combine_df.columns[2:])

,month,industry,pct_mom,pct_yoy
9,2023-07-31,WHOLE INDUSTRY,101.75,103.92
10,2023-07-31,Mining and quarying,100.23,98.07
11,2023-07-31,Mining of coal and lignite,98.17,94.19
12,2023-07-31,Extraction of crude petroleum and natural gas,105.97,94.71
13,2023-07-31,Mining of metal ores,99.22,110.91
...,...,...,...,...
41,2024-11-30,"Electricity, gas, steam and air conditioning s...",105.74,96.84
42,2024-11-30,"Water supply; sewerage, waste management and r...",111.79,97.21
43,2024-11-30,"Water collection, treatment and supply",104.33,101.48
44,2024-11-30,Sewerage and sewer treatment activities,98.69,99.99


In [ ]:
# Process data
check_values = False

with open("sheetnames.json") as js:
    sheet_cols = json.load(js)

# Create a function to streamline the process

number_of_sheets = 19

# Process monthly data
# for sheet_index in tqdm(range(0, number_of_sheets)):
for sheet_index in [0]:
    try:
        combine_df = pd.DataFrame()

        if sheet_index == 15:
            row_adj = 1
        else:
            row_adj = 0
        for excel_path in monthly_files:
            sheet = use_columns(excel_path, sheet_index).iloc[1 + row_adj:,:]
            combine_df = pd.concat([combine_df, sheet], axis = 0)

        # Custom conditions
        if sheet_index in [0, 10, 11, 13, 14, 15, 16, 17, 18]: # Sheets that need to merge multiple name columns to make sense
            if sheet_index == 15:
                ncol = 3
                bad_label = combine_df.iloc[:,1].astype(str).str.contains(r"Of which:", regex = True, flags = re.IGNORECASE)
                combine_df.loc[bad_label, 1] = None 
            else:
                ncol = 2
            if sheet_index in [16, 17]:
                is_parent = combine_df.iloc[:,0].str.startswith("By ", na=False)
                combine_df["name"] = (
                    (combine_df.iloc[:,0].where(is_parent).ffill()).astype(str) + 
                    " - " + 
                    combine_df.iloc[:,0].astype(str)
                ).str.replace(r"nan", "", regex=True).replace(r"^\s-\s", "", regex=True).replace("", None)
            else:
                combine_df = combine_columns(combine_df, ncol)
            var_list = ["month", "name"]
        else:
            var_list = ["month"]
        if sheet_index == 2:
            adjust = 1
        else:
            adjust = 0

        var_list.extend(list(sheet_cols["monthly"][f"{sheet_index}"]["columns"].values()))

        combine_df = combine_df.rename(columns=dict(
            list(zip(
                list(map(int, sheet_cols["monthly"][f"{sheet_index}"]["columns"].keys())),
                list(sheet_cols["monthly"][f"{sheet_index}"]["columns"].values())
            ))
        ))[var_list] # Rearranging the columns

        if check_values == False:
            combine_df = clean_data(
                combine_df,
                var_list[1],
                combine_df.columns[2 + adjust:]
            )
            combine_df.to_csv(os.path.join("combined_data", "monthly_data", f"{sheet_cols["monthly"][f"{sheet_index}"]["sheet"]}.csv", index=False))
            # combine_df.to_csv(f"combined_data/{sheet_cols[f"{sheet_index}"]["sheet"]}.csv", index=False)
    except Exception as error:
        print(f"Error at {sheet_index}: {error}")

KeyError: '0'

In [69]:
sheet_cols[f"{sheet_index}"]["columns"].values()

KeyError: '0'

In [ ]:
# # Combine into 1 Excel file
# if check_values == False:
#     with pd.ExcelWriter("macro_data.xlsx", engine = "openpyxl") as writer:
#         for csv_file in tqdm(os.listdir("combined_data")):
#             if csv_file.endswith(".csv"):
#                 df = pd.read_csv(os.path.join("combined_data", csv_file), index_col=False)
#                 sheet_name = os.path.splitext(csv_file)[0]
#                 df.to_excel(writer, sheet_name = sheet_name, index = False)

In [ ]:
# # Agriculture
# combined_agriculture = pd.DataFrame()

# for excel_path in monthly_files:
#     sheet = use_columns(excel_path, 0)
#     combined_agriculture = pd.concat([combined_agriculture, sheet], axis = 0)

# combined_agriculture.iloc[:,0] = combined_agriculture.iloc[:,0].ffill()

# combined_agriculture = combined_agriculture.assign(
#     product = combined_agriculture.iloc[:,0].astype(str) + " - " + combined_agriculture.iloc[:,1].fillna("")
# ).rename(columns={
#     2:"thousand_ha_yoy",
#     3:"thousand_ha_current",
#     4:"pct_yoy"
# })[["month", "product","thousand_ha_yoy", "thousand_ha_current", "pct_yoy"]] # Rearranging the columns

# combined_agriculture = clean_data(
#     combined_agriculture,
#     "product",
#     combined_agriculture.columns[2:]
# )

# if check_values == False:
#     combined_agriculture.to_csv("combined_data/combined_agr.csv", index=False)

In [ ]:
# # IIP

# combined_iip = pd.DataFrame()

# for excel_path in monthly_files:
#     sheet = use_columns(excel_path, 1)
#     combined_iip = pd.concat([combined_iip, sheet], axis = 0)
# combined_iip = combined_iip.rename(columns={
#     0:"industry",
#     1:"pct_yoy_lag1",
#     2:"pct_mom",
#     3:"pct_yoy",
#     4:"accum_yoy"
# })[["month", "industry", "pct_yoy_lag1", "pct_mom", "pct_yoy", "accum_yoy"]]

# if check_values == False:
#     combined_iip = clean_data(
#         combined_iip, 
#         "industry", 
#         combined_iip.columns[2:]
#     )
#     combined_iip.to_csv("combined_data/combined_iip.csv", index=False)

# clean_data(
#         combined_iip, 
#         "industry", 
#         combined_iip.columns[2:]
#     )

In [ ]:
# # Key Industrial products
# combined_kip = pd.DataFrame()

# for excel_path in monthly_files:
#     sheet = use_columns(excel_path, sheet_index = 2)
#     combined_kip = pd.concat([combined_kip, sheet], axis = 0)

# # Rearrange "month" to be the first column
# combined_kip = combined_kip.rename(columns={
#     0:"product",
#     1:"unit",
#     2:"performance_lag1",
#     3:"est_performance",
#     4:"accum_performance",
#     5:"pct_yoy",
#     6:"pct_accum_yoy"
# })[["month", "product", "unit", "performance_lag1", "est_performance", "accum_performance", "pct_yoy", "pct_accum_yoy"]]

# if check_values == False:
#     combined_kip = clean_data(
#         combined_kip, 
#         "product", 
#         combined_kip.columns[3:]
#     )
#     combined_kip.to_csv("combined_data/combined_kip.csv", index=False)

# clean_data(
#     combined_kip, 
#     "product", 
#     combined_kip.columns[3:]
# )

In [ ]:
# # Labour Employed Index - LEI - BY INDUSTRY
# combined_lei_ind = pd.DataFrame()

# for excel_path in monthly_files:
#     sheet = use_columns(excel_path, sheet_index = 3)
#     combined_lei_ind = pd.concat([combined_lei_ind, sheet], axis = 0)

# combined_lei_ind = combined_lei_ind.rename(columns={
#     0:"industry",
#     1:"ind_lei_mom",
#     2:"ind_lei_yoy"
# })[["month", "industry", "ind_lei_mom", "ind_lei_yoy"]]

# if check_values == False:
#     combined_lei_ind = clean_data(
#         combined_lei_ind,
#         "industry",
#         combined_lei_ind.columns[2:]
#     )
#     combined_lei_ind.to_csv("combined_data/combined_lei_ind.csv", index=False)

# clean_data(
#     combined_lei_ind,
#     "industry",
#     combined_lei_ind.columns[2:]
# )

In [ ]:
# # Labour Employed Index - LEI - BY PROVINCE
# combined_lei_prov = pd.DataFrame()

# for excel_path in monthly_files:
#     sheet = use_columns(excel_path, sheet_index = 4)
#     combined_lei_prov = pd.concat([combined_lei_prov, sheet], axis = 0)

# combined_lei_prov = combined_lei_prov.rename(columns={
#     0:"province",
#     1:"prov_lei_mom",
#     2:"prov_lei_yoy"
# })[["month", "province", "prov_lei_mom", "prov_lei_yoy"]]

# if check_values == False:
#     combined_lei_prov = clean_data(
#         combined_lei_prov,
#         "province",
#         combined_lei_prov.columns[2:]
#     )
#     combined_lei_prov.to_csv("combined_data/combined_lei_prov.csv", index=False)
# clean_data(
#     combined_lei_prov,
#     "province",
#     combined_lei_prov.columns[2:]
# )

In [ ]:
# # Some indicators about enterprise

# combined_ei = pd.DataFrame()

# for excel_path in monthly_files:
#     sheet = use_columns(excel_path, sheet_index = 5)
#     combined_ei = pd.concat([combined_ei, sheet], axis = 0)

# combined_ei = combined_ei.rename(columns={
#     0: "indicator",
#     1: "value_lag1",
#     2: "value",
#     3: "accum_value",
#     4: "value_mom",
#     5: "value_yoy",
#     6: "accum_yoy"
# })[["month", "indicator", "value_lag1", "value", "accum_value", "value_mom", "value_yoy", "accum_yoy"]]

# if check_values == False:
#     combined_ei = clean_data(
#         combined_ei,
#         "indicator",
#         combined_ei.columns[2:]
#     )
#     combined_ei.to_csv("combined_data/combined_ei.csv", index=False)

# clean_data(
#     combined_ei,
#     "indicator",
#     combined_ei.columns[2:]
# )

In [ ]:
# # New Registered Enterprises

# combined_new_e = pd.DataFrame()

# for excel_path in monthly_files:
#     sheet = use_columns(excel_path, sheet_index = 6).drop(0, axis=1)
#     combined_new_e = pd.concat([combined_new_e, sheet], axis = 0)

# # This data does not seem to have monthly data, only accummulative data
# combined_new_e = combined_new_e.rename(columns={
#     1: "industry",
#     2: "accum_new_enterprise",
#     3: "accum_capital_bil",
#     4: "accum_employee",
#     5: "accum_new_yoy",
#     6: "accum_cap_yoy",
#     7: "accum_emp_yoy"
# })[[
#     "month",
#     "industry",
#     "accum_new_enterprise",
#     "accum_capital_bil",
#     "accum_employee",
#     "accum_new_yoy",
#     "accum_cap_yoy",
#     "accum_emp_yoy"
# ]]

# if check_values == False:
#     combined_new_e = clean_data(
#         combined_new_e,
#         "industry",
#         combined_new_e.columns[2:]
#     )
#     combined_new_e.to_csv("combined_data/combined_new_e.csv", index=False)

# clean_data(
#     combined_new_e,
#     "industry",
#     combined_new_e.columns[2:]
# )

In [ ]:
# # Enterprises returned

# combined_return_e = pd.DataFrame()

# for excel_path in monthly_files:
#     sheet = use_columns(excel_path, sheet_index = 7)
#     combined_return_e = pd.concat([combined_return_e, sheet], axis = 0)

# # This data does not seem to have monthly data, only accummulative data
# combined_return_e = combined_return_e.rename(columns={
#     0: "industry",
#     1: "accum_return_ent_yoy",
#     2: "accum_return_ent",
#     3: "return_ent_pct_yoy"
# })[["month", "industry", "accum_return_ent_yoy", "accum_return_ent", "return_ent_pct_yoy"]]

# if check_values == False:
#     combined_return_e = clean_data(
#         combined_return_e,
#         "industry",
#         combined_return_e.columns[2:]
#     )
#     combined_return_e.to_csv("combined_data/combined_return_e.csv", index=False)

# clean_data(
#     combined_return_e,
#     "industry",
#     combined_return_e.columns[2:]
# )

In [ ]:
# # Enterprises temporary ceased

# combined_ceased_e = pd.DataFrame()

# for excel_path in monthly_files:
#     sheet = use_columns(excel_path, sheet_index = 8)
#     combined_ceased_e = pd.concat([combined_ceased_e, sheet], axis = 0)

# # This data does not seem to have monthly data, only accummulative data
# combined_ceased_e = combined_ceased_e.rename(columns={
#     0: "industry",
#     1: "accum_ceased_ent_yoy",
#     2: "accum_ceased_ent",
#     3: "ceased_ent_pct_yoy"
# })[["month", "industry", "accum_ceased_ent_yoy", "accum_ceased_ent", "ceased_ent_pct_yoy"]]

# if check_values == False:
#     combined_ceased_e = clean_data(
#         combined_ceased_e,
#         "industry",
#         combined_ceased_e.columns[2:]
#     )
#     combined_ceased_e.to_csv("combined_data/combined_ceased_e.csv", index=False)

# clean_data(
#     combined_ceased_e,
#     "industry",
#     combined_ceased_e.columns[2:]
# )

In [ ]:
# # Enterprises complete dissolution

# combined_diss_e = pd.DataFrame()

# for excel_path in monthly_files:
#     sheet = use_columns(excel_path, sheet_index = 9)
#     combined_diss_e = pd.concat([combined_diss_e, sheet], axis = 0)

# # This data does not seem to have monthly data, only accummulative data
# combined_diss_e = combined_diss_e.rename(columns={
#     0: "industry",
#     1: "accum_diss_ent_yoy",
#     2: "accum_diss_ent",
#     3: "diss_ent_pct_yoy"
# })

# if check_values == False:
#     combined_diss_e = clean_data(
#         combined_diss_e,
#         "industry",
#         combined_diss_e.columns[2:]
#     )
#     combined_diss_e.to_csv("combined_data/combined_diss_e.csv", index=False)

In [ ]:
# # Realized investment under State budget

# combined_invest = pd.DataFrame()

# for excel_path in monthly_files:
#     sheet = use_columns(excel_path, sheet_index = 10, col_index = [1, 2, 3, 4, 5, 6])
#     combined_invest = pd.concat([combined_invest, sheet], axis = 0)

# combined_invest = combined_invest.rename(columns={
#     1:"area",
#     2:"inv_bil_lag1",
#     3:"inv_bil",
#     4:"accum_inv",
#     5:"accum_inv_to_plan",
#     6:"accum_inv_yoy"
# })[["month", "area", "inv_bil_lag1", "inv_bil", "accum_inv", "accum_inv_to_plan", "accum_inv_yoy"]]

# if check_values == False:
#     combined_invest = clean_data(
#         combined_invest,
#         "area",
#         combined_invest.columns[2:]
#     )
#     combined_invest.to_csv("combined_data/combined_invest.csv", index=False)

# clean_data(
#     combined_invest,
#     "area",
#     combined_invest.columns[2:]
# )

In [ ]:
# # FDI

# combined_fdi = pd.DataFrame()

# for excel_path in monthly_files:
#     sheet = use_columns(excel_path, sheet_index = 11, col_index = [1, 2, 3, 4])
#     combined_fdi = pd.concat([combined_fdi, sheet], axis = 0)

# # This data does not seem to have monthly data, only accummulative data
# combined_fdi = combined_fdi.rename(columns={
#     1: "area",
#     2: "n_projects",
#     3: "new_registered_cap_mil_usd",
#     4: "reg_cap_adjust_mil_usd"
# }).assign(
#     n_projects = lambda df : df["n_projects"].fillna(0),
#     new_registered_cap_mil_usd = lambda df : df["new_registered_cap_mil_usd"].fillna(0),
#     reg_cap_adjust_mil_usd = lambda df : df["reg_cap_adjust_mil_usd"].fillna(0),
# )[["month", "area", "n_projects", "new_registered_cap_mil_usd", "reg_cap_adjust_mil_usd"]]

# if check_values == False:
#     combined_fdi = clean_data(
#         combined_fdi,
#         "area",
#         combined_fdi.columns[2:]
#     )
#     combined_fdi.to_csv("combined_data/combined_fdi.csv", index=False)

In [ ]:
# # Retail sale

# combined_retail = pd.DataFrame()

# for excel_path in monthly_files:
#     sheet = use_columns(excel_path, sheet_index = 12)
#     combined_retail = pd.concat([combined_retail, sheet], axis = 0)

# combined_retail = combined_retail.rename(columns={
#     0:"goods_service",
#     1:"perf_bil_vnd_lag1",
#     2:"est_perf_bil_vnd",
#     3:"accum_bil_vnd",
#     4:"structure_pct",
#     5:"pct_yoy",
#     6:"pct_accum_yoy"
# })[["month","goods_service", "perf_bil_vnd_lag1", "est_perf_bil_vnd", "accum_bil_vnd", "structure_pct", "pct_yoy", "pct_accum_yoy"]]

# if check_values == False:
#     combined_retail = clean_data(
#         combined_retail,
#         "goods_service",
#         combined_retail.columns[2:]
#     )
#     combined_retail.to_csv("combined_data/combined_retail.csv", index=False)

In [ ]:
# # Export

# combined_export = pd.DataFrame()

# for excel_path in monthly_files:
#     sheet = use_columns(excel_path, sheet_index = 13, col_index = [1, 2, 3, 5, 6, 8, 9, 11, 12])
#     combined_export = pd.concat([combined_export, sheet], axis = 0)

# combined_export = combined_export.rename(columns={
#     1:"product",
#     2:"est_vol_1000_ton",
#     3:"est_val_mil_usd",
#     5:"est_accum_vol_1000_ton",
#     6:"est_accum_val_mil_usd",
#     8:"yoy_vol_1000_ton",
#     9:"yoy_val_mil_usd",
#     11:"yoy_accum_vol_1000_ton",
#     12:"yoy_accum_val_mil_usd"
# })[["month", "product", "est_vol_1000_ton", "est_val_mil_usd", "est_accum_vol_1000_ton", "est_accum_val_mil_usd", "yoy_vol_1000_ton", "yoy_val_mil_usd", "yoy_accum_vol_1000_ton", "yoy_accum_val_mil_usd"]]

# if check_values == False:
#     combined_export = clean_data(
#         combined_export,
#         "product",
#         combined_export.columns[2:]
#     )
#     combined_export.to_csv("combined_data/combined_export.csv", index=False)

In [ ]:
# # Import

# combined_import = pd.DataFrame()

# for excel_path in monthly_files:
#     sheet = use_columns(excel_path, sheet_index = 14, col_index = [1, 2, 3, 5, 6, 8, 9, 11, 12])
#     combined_import = pd.concat([combined_import, sheet], axis = 0)

# combined_import = combined_import.rename(columns={
#     1:"product",
#     2:"est_vol_1000_ton",
#     3:"est_val_mil_usd",
#     5:"est_accum_vol_1000_ton",
#     6:"est_accum_val_mil_usd",
#     8:"yoy_vol_1000_ton",
#     9:"yoy_val_mil_usd",
#     11:"yoy_accum_vol_1000_ton",
#     12:"yoy_accum_val_mil_usd"
# })[["month", "product", "est_vol_1000_ton", "est_val_mil_usd", "est_accum_vol_1000_ton", "est_accum_val_mil_usd", "yoy_vol_1000_ton", "yoy_val_mil_usd", "yoy_accum_vol_1000_ton", "yoy_accum_val_mil_usd"]]

# if check_values == False:
#     combined_import = clean_data(
#         combined_import,
#         "product",
#         combined_import.columns[2:]
#     )
#     combined_import.to_csv("combined_data/combined_import.csv", index=False)

In [ ]:
# # CPI

# combined_cpi = pd.DataFrame()

# for excel_path in monthly_files:
#     sheet = use_columns(excel_path, sheet_index = 15).iloc[2:,:]
#     combined_cpi = pd.concat([combined_cpi, sheet], axis = 0)

# combined_cpi = combined_cpi.assign(
#     sector = lambda df : (df[0].ffill() + ": " + df[1].fillna("").astype(str) + " " + df[2].fillna("").astype(str)).str.replace("nan", "")
# )

# combined_cpi = combined_cpi.rename(columns={
#     3:"cpi_base",
#     4:"cpi_yoy",
#     5:"cpi_begin_year",
#     6:"cpi_mom",
#     7:"avg_cpi_yoy"
# })[["month", "sector","cpi_base", "cpi_yoy", "cpi_begin_year", "cpi_mom", "avg_cpi_yoy"]]

# if check_values == False:
#     combined_cpi = clean_data(
#         combined_cpi,
#         "sector",
#         combined_cpi.columns[2:]
#     )
#     combined_cpi.to_csv("combined_data/combined_cpi.csv", index=False)

# clean_data(
#     combined_cpi,
#     "sector",
#     combined_cpi.columns[2:]
# )

In [ ]:
# Carriage of passengers

# combined_car_pas = pd.DataFrame()

# for excel_path in monthly_files:
#     sheet = use_columns(excel_path, sheet_index = 16).iloc[1:,]
#     combined_car_pas = pd.concat([combined_car_pas, sheet], axis = 0)

# combined_car_pas = combined_car_pas.rename(columns={
#     1:"product",
#     2:"est_vol_1000_ton",
#     3:"est_val_mil_usd",
#     5:"est_accum_vol_1000_ton",
#     6:"est_accum_val_mil_usd",
#     8:"yoy_vol_1000_ton",
#     9:"yoy_val_mil_usd",
#     11:"yoy_accum_vol_1000_ton",
#     12:"yoy_accum_val_mil_usd"
# })[["month", "product", "est_vol_1000_ton", "est_val_mil_usd", "est_accum_vol_1000_ton", "est_accum_val_mil_usd", "yoy_vol_1000_ton", "yoy_val_mil_usd", "yoy_accum_vol_1000_ton", "yoy_accum_val_mil_usd"]]

# if check_values == False:
#     combined_car_pas = clean_data(
#         combined_car_pas,
#         "product",
#         combined_car_pas.columns[2:]
#     )
#     combined_car_pas.to_csv("combined_data/combined_car_pas.csv", index=False)


In [33]:
is_parent = combined_car_pas.iloc[:,0].str.startswith("By ", na=False)

combined_car_pas["name"] = (
    (combined_car_pas.iloc[:,0].where(is_parent).ffill()).astype(str) + 
    " - " + 
    combined_car_pas.iloc[:,0].astype(str)
).str.replace(r"nan", "", regex=True).replace(r"^\s-\s", "", regex=True).replace("", None)

In [ ]:
# # International visitors

# combined_visitors = pd.DataFrame()

# for excel_path in monthly_files:
#     sheet = use_columns(excel_path, sheet_index = 18, col_index = [1, 2, 3, 4, 5, 6])
#     combined_visitors = pd.concat([combined_visitors, sheet], axis = 0)

# combined_visitors = combined_visitors.rename(columns={
#     1:"category",
#     2:"arrivals_lag1",
#     3:"arrivals",
#     4:"accum_arrivals",
#     5:"arrivals_yoy",
#     6:"accum_arrivals_yoy"
# })[["month", "category", "arrivals_lag1", "arrivals", "accum_arrivals", "arrivals_yoy", "accum_arrivals_yoy"]]

# if check_values == False:
#     combined_visitors = clean_data(
#         combined_visitors,
#         "category",
#         combined_visitors.columns[2:]
#     )
#     combined_visitors.to_csv("combined_data/combined_visitors.csv", index=False)
